# Data Set Size vs. Regional Scatter
This notebook illustrates the difference between two possibilities of increasing
precision of the anomaly quantification: more data and less uncertainty (presumably
by modeling and removing effects).

In [ ]:
import numpy as np
from plotconfig import *
from cmcrameri.cm import *
from cache import cached_call
import matplotlib.pyplot as plt
from zeal2022hf import get_cm_colors
from pdtoolbox import gamma_pdf, normal_pdf
from reheatfunq.regional import default_prior
from reheatfunq.regional.backend import gamma_mle
from reheatfunq import HeatFlowAnomalyPosterior, AnomalyLS1980, GammaConjugatePrior

In [ ]:
MU = 75.0

In [ ]:
AnoPH = 10e6

In [ ]:
ano = AnomalyLS1980(np.array([(0, -80e3), (0, 80e3)]), 14e3)

In [ ]:
PSNV = np.loadtxt('results/05-GCP-Parameters.txt',delimiter=',')

In [ ]:
def gamma_parameters(mu, sigma):
    theta = (sigma * sigma) / mu
    k = mu / theta
    return k, theta

In [ ]:
def generate_regional_sample(N, sigma, rng, ano, PH=AnoPH, mu=MU):
    # Parameter combination to achieve mu and sigma:
    k,theta = gamma_parameters(mu, sigma)
    Q = np.sort(rng.gamma(k, size=N) * theta)
    x = 160e3 * (rng.random(size=N) - 0.5)
    y = 160e3 * (rng.random(size=N) - 0.5)
    xy = np.zeros((x.size,2))
    xy[:,0] = x
    xy[:,1] = y
    q = Q +  1e3*AnoPH*ano(xy)
    return xy, q

In [ ]:
def analyze_regional_sample(xy, q, ano, gcp, dmin=0.0):
    hfp = HeatFlowAnomalyPosterior(q, xy[:,0], xy[:,1], ano, gcp, dmin)
    return hfp.tail_quantiles([0.9, 0.5, 0.1])

In [ ]:
def monte_carlo(N, sigma, M, dmin=0.0, ano=ano, seed=29189, p=PSNV[0], s=PSNV[1], n=PSNV[2], v=PSNV[3],
                PH=AnoPH, mu=MU):
    rng = np.random.default_rng(seed)
    gcp = GammaConjugatePrior(p, s, n, v)
    res = np.empty((M,3))
    for i in range(M):
        xy, q = generate_regional_sample(N, sigma, rng, ano, PH, mu)
        res[i] = analyze_regional_sample(xy, q, ano, gcp, dmin)
    
    return res

Two example distribution. We start from the blue, wider one, parameterized by `k0` and `theta0`.

In [ ]:
k0, theta0 = gamma_parameters(MU, 20.0)
k1, theta1 = gamma_parameters(MU, 3.0)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
qplot = np.linspace(1e-3, 180.0, 500)
yplot = gamma_pdf(qplot, k0, theta0)
ax.plot(qplot, yplot)
yplot = gamma_pdf(qplot, k1, theta1)
ax.plot(qplot, yplot)
ax.set_ylim(0, ax.get_ylim()[1])

Perform simulations:

In [ ]:
SIGMA = np.linspace(1, 20, 20)
SIGMA = np.concatenate(([1e-2],SIGMA))
N = np.round(np.geomspace(20,1000, 20)).astype(int)
N_MC = 200

In [ ]:
res_SIGMA = np.empty((SIGMA.size, N_MC, 3))
res_N = np.empty((N.size, N_MC, 3))

In [ ]:
for i,s in enumerate(SIGMA):
    print(i,"/",len(SIGMA),":",s)
    res_SIGMA[i,:,:] = cached_call(monte_carlo, 20, s, N_MC)

In [ ]:
for i,n in enumerate(N):
    print(i,"/",len(N),":",n)
    res_N[i,:,:] = cached_call(monte_carlo, n, 20., N_MC)

In [ ]:
colors = get_cm_colors(vik, 13)
color0 = colors[0]
color1 = colors[5]
color2 = colors[12]
color3 = colors[7]

In [ ]:
fig = plt.figure(figsize=(5.8, 2.3))
#ax_bg = fig.add_axes((0,0,1,1))
ax = fig.add_axes((0.09, 0.16, 0.4, 0.82))
ax.fill_between(SIGMA, 1e-6*np.median(res_SIGMA[:,:,0], axis=1),
                1e-6*np.median(res_SIGMA[:,:,2], axis=1), color=color1,
                label='80 % quantile\n(symmetric)')
ax.plot(SIGMA, 1e-6*np.median(res_SIGMA[:,:,1], axis=1), color=color0,
        label='Median')
ax.axhline(1e-6*AnoPH, color='k', linewidth=0.7,linestyle='--')
ax.set_xlabel('$\sigma$ ($\mathrm{mW\,m}^{-2}$)', labelpad=2)
ax.set_ylabel('Frictional power $P_H$ (MW)')
ax.text(0.5, 125, '(a)')
ax.set_xlim(0,20)
ax.legend(loc='upper center', bbox_to_anchor=(0.36, 1.0))

ax = fig.add_axes((0.59, 0.16, 0.4, 0.82))
ax.fill_between(N, 1e-6*np.median(res_N[:,:,0], axis=1),
                1e-6*np.median(res_N[:,:,2], axis=1), color=color3,
                label='80 % quantile\n(symmetric)')
ax.plot(N, 1e-6*np.median(res_N[:,:,1], axis=1), color=color2,
        label='Median')
ax.axhline(1e-6*AnoPH, color='k', linewidth=0.7,linestyle='--')
ax.set_ylabel('Frictional power $P_H$ (MW)')
ax.set_xscale('log')
ax.set_xlabel('$N$', labelpad=2)
ax.text(21.8, 125, '(b)')
ax.set_xlim(20, 1000)
ax.legend()

fig.savefig('figures/A8-Precision-Sigma-N.pdf')

### License
```
A notebook to determine investigate the dependence of heat flow
anomaly quantification on regional aggregate heat flow variance
and the sample size.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2022 Malte J. Ziebarth
            

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```